In [1]:
import pandas as pd
import json

from geopy.geocoders import Nominatim
import folium

import time

### (1) create json

Ask claude: Search for the capital & zipcode for 50 states in US


### (2)	valid json file and capital address(USPS)
1.JSON

https://jsonlint.com/

https://codebeautify.org/jsonvalidator

In [2]:
with open('us_state_capitals.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

2.address

https://tools.usps.com/zip-code-lookup.htm

https://www.census.gov/data.html

In [4]:
geolocator = Nominatim(user_agent="address_verifier")
ls_verify = []

for _, row in df.iterrows():
    try:
        location = geolocator.geocode(row['address'], country_codes="us")
        if location:
            print(f"Address found: {row['address']}")
            ls_verify.append(None)
        else:
            print(f"Address not found: {row['address']}")
            ls_verify.append("Address not found")
    except:
        print(f"Error processing address: {row['address']}")
        ls_verify.append("Error processing address")

    time.sleep(10)  # To avoid hitting the API too quickly
df['verification'] = ls_verify

Address found: 600 Dexter Ave, Montgomery, AL
Address found: 120 4th St, Juneau, AK
Address found: 1700 W Washington St, Phoenix, AZ
Address found: 500 Woodlane St, Little Rock, AR
Address found: 1315 10th St, Sacramento, CA
Address found: 200 E Colfax Ave, Denver, CO
Address found: 210 Capitol Ave, Hartford, CT
Address found: 411 Legislative Ave, Dover, DE
Address found: 400 S Monroe St, Tallahassee, FL
Address found: 206 Washington St SW, Atlanta, GA
Address found: 415 S Beretania St, Honolulu, HI
Address found: 700 W Jefferson St, Boise, ID
Error processing address: 401 S 2nd St, Springfield, IL
Address found: 200 W Washington St, Indianapolis, IN
Address found: 1007 E Grand Ave, Des Moines, IA
Address found: 300 SW 10th Ave, Topeka, KS
Address found: 700 Capitol Ave, Frankfort, KY
Address found: 900 N 3rd St, Baton Rouge, LA
Address found: 210 State St, Augusta, ME
Address found: 100 State Circle, Annapolis, MD
Address found: 24 Beacon St, Boston, MA
Address found: 100 N Capitol Av

### (3) provide longtitude and latitude

In [5]:
latitudes = []
longitudes = []

geolocator = Nominatim(user_agent="geoapi")

for _, row in df.iterrows():
    try:
        location = geolocator.geocode(f"{row['capital']}, {row['state']}, USA")

        try:
            latitude = location.latitude
            longitude = location.longitude
            # print(f"{capital}: ({location.latitude}, {location.longitude})")
        except:
            print(f"Error retrieving coordinates for {row['capital']}")
            latitude = eval(input("Enter latitude of {}: ".format(row['capital'])))
            longitude = eval(input("Enter longitude of {}: ".format(row['capital'])))
    except:
        print(f"{row['capital']}: Not found")
        latitude = eval(input("Enter latitude of {}: ".format(row['capital'])))
        longitude = eval(input("Enter longitude of {}: ".format(row['capital'])))

    latitudes.append(latitude)
    longitudes.append(longitude)
    time.sleep(10)

In [6]:
df['latitude'] = latitudes
df['longitude'] = list(longitudes)

In [7]:
df.to_json('us_state_capitals_with_coordinates.json', orient='records',indent=4,index=False)

### (4)	valid json file
https://jsonlint.com/

https://codebeautify.org/jsonvalidator

In [8]:
with open('us_state_capitals_with_coordinates.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

### (5)	verify longtitude and latitude
website: https://www.google.com/maps?q=38.8977,-77.0365 (change the numbers)

In [9]:
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

In [10]:
for _, row in df.iterrows():

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

    folium.map.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'''<div style="font-size: 10pt; color: black;">{row['state']}<br>{row['capital']}</div>'''
        )
    ).add_to(m)

In [11]:
m.save("us_capitals_check.html")